In [ ]:
#load packages
import pandas as pd
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
from typing import Union, List, Tuple
import json
from tqdm import tqdm 
from peft import PeftModel
from datasets import Dataset

In [ ]:
#try again with full model


In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Load full precision model (deterministic)
model = AutoModelForSequenceClassification.from_pretrained(
    "/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2",
    device_map = 'auto'
)
model.eval()

tokenizer = AutoTokenizer.from_pretrained("/work/MarkusLundsfrydJensen#1865/test_models_save/Option_2")
# Move to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

print("Model loaded successfully!")

# Same predict function as before
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.softmax(logits, dim=-1)
        prediction = torch.argmax(probs, dim=-1)
    
    return {
        'prediction': prediction.item(),
        'probability_class_0': probs[0][0].item(),
        'probability_class_1': probs[0][1].item()
    }

/work/MarkusLundsfrydJensen#1865/miniconda3/envs/blame_bert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-10-29 11:49:22.859912: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-29 11:49:22.926017: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-29 11:49:24.297223: I tensorflow/core/util/port.cc:153] oneD

Model loaded successfully!


In [3]:
import json
json_path = "/work/MarkusLundsfrydJensen#1865/Bachelor_project/Model_data/validation_set.json"

with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

for entry in data:
    text = entry["text"]

    result = predict(text)

    entry["pred"] = result["prediction"]

true_labels = 0
pred_true = 0
false_labels = 0
pred_false = 0

correct_pred = 0
incorrect_pred = 0

for entry in data:
    if entry["label"] == 1:
        true_labels += 1
    if entry["pred"] == 1:
        pred_true +=1

    if entry["label"] == 0:
        false_labels += 1
    if entry["pred"] == 0:
        pred_false +=1

    if entry["label"] == entry["pred"]:
        correct_pred +=1

    if entry["label"] != entry["pred"]:
        incorrect_pred +=1


print(correct_pred/len(data))

0.7635658914728682


In [5]:
pred_true

118